In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from pathlib import Path
import unidecode
import pandas as pd

import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
import constants

In [4]:
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 250

In [5]:
df = pd.read_csv("../data/kanunum-nlp-doc-analysis-dataset.csv")

In [6]:
df.shape

(4142, 16)

### Alanlar
- **id**                    : İçerik kimliği
- **kategori**              : İçerik Türü/Kategorisi
- **baslik**                : İçerik Başlığı
- **rega_no**               : İçeriğin yayınlandığı Resmi Gazete Sayısı
- **mukerrer_no**           : İçeriğin yayınlandığı Resmi Gazete Mükerrer Numarası (0 ise mükerrer olmadığını belirtir)
- **rega_tarihi**           : İçeriğin yayınlandığı Resmi Gazete Tarihi (`date:yyyy-mm-dd` formatında)
- **kurum**                 : İçeriği(Genelge, Cumhurbaşkanlığı(CB) Kararnamesi, Tüzük, Yönetmelik, Tebliğ) çıkaran kurum adı
- **mevzuat_no**            : İçeriğin ilgili numarası (Detaylar için ilgili kategoriye bakınız) 
- **belge_sayi**            : Genelge üzerindeki Sayı ifadesinin değeri (sadece işlenebilenler)
- **mevzuat_tarihi**        : İçeriğin ilgili Tarihi (`date:yyyy-mm-dd` formatında)
- **donem**                 : Komisyon Raporunun dönemini (örn: "27. Dönem")
- **sira_no**               : Komisyon Raporunun Sıra Sayısı (örn: "230", "42 ek 1" vb.)
- **madde_sayisi**          : İçeriğin(Kanun, KHK, CB Kararnamesi, Tüzük, Yönetmelik) bölümlerini (Madde sayısını ve işlenemeyen hükümleri/maddeleri, _madde metinlerinin_ içindeki "Madde <sayi> - ..." ifadeleri hariç )
- **data_text**             : İçerik metni (Büyük/Küçük Harf, Noktalama ve "\n" içermektedir)
- **url**                   : İçerik metninin esas alındığı websayfasının linki (kaynak linkler güncel olmayabilir)
- **kanunum_url**           : İçeriğin Kanunum linki

**İçerik Başlıkları**, Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Tebliğ, Yönetmelik ve Tebliğ için içeriğin adı olan başlık kullanılırken Komisyon Raporu, Genelge ve Özelgelerde tanımlayıcı olmak üzere işelenerek üretilmiş bir başlık tercih edilmiştir.

**Kurum Adı**, Kanunum veritabanına işlenmiş hali ile kullanılmaktadır. Örneğin; Üniversite Yönetmeliklerinde kurum adı ayrı ayrı üniversite adı kullanmak yerine tek bir ad ("Üniversite") olarak işlenmiştir. Kurum tespiti yapılırken Bakanlık isimlerinin değişmiş olabileceği ayrıca dikkate alınmalıdır.

## Kategoriler
 Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Genelge, Tebliğ, Yönetmelik, Komisyon Raporu, Tüzük ve Özelge olmak üzere toplam 10 kategori bulunmaktadır.

Tüm içerikler için `kategori`, `baslik`, `data_text`, `url`, `kanunum_url` alanları sunulmuş olup `url` alanı içerik metninin esas alındığı sayfa linkini belirtmektedir. Diğer dikkate alınacak alanlar aşağıdaki gibi olup detaylar alt başlıklarda verilmiştir.


| Kategori                                                               | İçerik Sayısı     | İlgili alanlar (Üst Veriler)                                                                                              |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, data_text, url                                                       |
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | kategori, baslik, donem, sira_no, data_text, url                                                                          |
| [Genelge](#kategori-genelge)                                           | 385               | kategori, baslik, kurum, mevzuat_no, belge_sayi, mevzuat_tarihi, data_text, url                                           |
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tüzük](#kategori-tüzük)                                               | 82                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, data_text, url                                                |
| [Özelge](#kategori-özelge)                                             | 750               | kategori, baslik, kurum, mevzuat_tarihi, data_text, url M

### BELGE SAYISI

In [7]:
belge_sayisi_extractor = BelgeSayisiExtractor()

Yarisma submission modu: KAPALI


In [8]:
belge_sayisi_extractor.extract(df)

'belge_sayi' cikariliyor... -> 'belge_sayi_pred'


  0%|          | 0/385 [00:00<?, ?it/s]

In [9]:
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=True)

Veri Sayisi: 385
Dogru Tahmin Sayisi: 86
Yanlis Tahmin Sayisi: 299
Basari Orani: 0.2234


In [10]:
# NaN olmayan etiketlerde %100 başarı
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 43
Dogru Tahmin Sayisi: 43
Yanlis Tahmin Sayisi: 0
Basari Orani: 1.0


In [11]:
show_false_predictions(input_df=df,
                       feature_extractor=belge_sayisi_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=True,
                       first_n=10)

,belge_sayi,belge_sayi_pred,baslik,url,kategori,data_text
917,undefined,B.07.0.GEL.0.44/4412-17-047504,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/5",http://www.gib.gov.tr/node/89371,Genelge,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/47504\n\nTarih 15/12/1998\n\nSayı B.07.0.GEL.0.44/4412-17-047504\n\nKapsam \n\nT.C\n\nMALIYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.44/4412-17-047504 \n\nKONU:GELİR VERGİSİ KANU...
918,undefined,B.07.0.GEL.0.71/7100-89 - 009461,"GELİR İDARESİ BAŞKANLIĞI, VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2",http://www.gib.gov.tr/node/89248,Genelge,VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2\n\nTarih 08/03/2003\n\nSayı B.07.0.GEL.0.71/7100-89 - 009461\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.71/7100-89\n\nTARİH : 08.03.2003\n\nU...
919,undefined,B. 13.2.SGK.0.71.09.00/53>~? 3 0 MAYİS 2Oİ2,"SOSYAL GÜVENLİK KURUMU, GENELGE 2012-20",https://kms.kaysis.gov.tr/Home/Goster/73822,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nİnsan Kaynaklan Daire Başkanlığı\n\nSayı : B. 13.2.SGK.0.71.09.00/53>~? 3 0 MaYIS 2ÖÎ2\nKonu : Genelge Değişikliği\n\nGENELGE\n2012/20\n\n07/01/2010 tarihli ve 2010/3 sayılı “ Mal Bildirimi (e-mal bildi...
922,undefined,B.13.2.SSK.5.01.08.00/Vİİİ-031-502522 ANKARA,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/106310,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSosyal Sigortalar Kurumu Başkanlığı (Devredilen)\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B.13.2.SSK.5.01.08.00/VIII-031-502522 ANKARA\nKONU : Yatırımların ve istihdam...
923,undefined,B. 13.2.SSK.5.01.08.00/Vİİİ-031-946961 ANKARA,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/128284,Genelge,T.C.\nSOSYAL SİGORTALAR KURUMU BAŞKANLIĞI \n\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B. 13.2.SSK.5.01.08.00/VIII-031-946961 ANKARA\nKONU : ihale Konusu işlerden Kaynaklanan 06/12/2005\n\nBorçların Takibi\n\nGEN...
924,undefined,B.07.0.GEL.0.40/4085-017967,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO : 2004/1",http://www.gib.gov.tr/node/89196,Genelge,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO : 2004/1\n\nTarih 22/04/2004\n\nSayı B.07.0.GEL.0.40/4085-017967\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI :B.07.0.GEL.0.40/4085-\n\nKONU :GELİR VERGİSİ KANUNU İÇ GENELGES...
925,undefined,B.07.0.GEL.0.40/4085-59/031997,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO : 2003/3",http://www.gib.gov.tr/node/89361,Genelge,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO : 2003/3\n\nTarih 22/07/2003\n\nSayı B.07.0.GEL.0.40/4085-59/031997\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.40/4085-59\n\nKONU : GELİR VERGİSİ KANUNU İÇ G...
926,undefined,B.07.0.GEL.0.49/4938-67-19962,"GELİR İDARESİ BAŞKANLIĞI, KURUMLAR VERGİSİ İÇ GENELGESİ SERİ NO 2000/1",http://www.gib.gov.tr/node/89333,Genelge,KURUMLAR VERGİSİ İÇ GENELGESİ SERİ NO 2000/1\n\nTarih 27/04/2000\nSayı B.07.0.GEL.0.49/4938-67-19962\nKapsam \n\nTC.MALİYE BAKANLIĞI \nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.49/4938-67-19962\nKONU : KURUMLAR VERGİSİ İÇ GENELGESİ SERİ NO:...
927,undefined,B.07.1.GİB.0.03.50/5038-1-78000,"GELİR İDARESİ BAŞKANLIĞI, KURUMLAR VERGİSİ İÇ GENELGESİ SERİ NO 2009/2",http://www.gib.gov.tr/node/89212,Genelge,KURUMLAR VERGİSİ İÇ GENELGESİ SERİ NO 2009/2\n\nTarih 13/08/2009\nSayı B.07.1.GİB.0.03.50/5038-1-78000\nKapsam \n\nT.C.MALİYE BAKANLIĞI\nGelir İdaresi Başkanlığı\n\nSayı : B.07.1.GİB.0.03.50/5038-1-78000\n\nKonu :5520 SAYILI KURUMLAR VERGİSİ İÇ G...
928,undefined,B.07.1.GİB.0.03.50/5038-78002,"GELİR İDARESİ BAŞKANLIĞI, KURUMLAR VERGİ

### MADDE SAYISI

In [23]:
madde_sayisi_extractor = MaddeSayisiExtractor()

Yarisma submission modu: KAPALI


In [24]:
madde_sayisi_extractor.extract(df)

'madde_sayisi' cikariliyor... -> 'madde_sayisi_pred'


  0%|          | 0/1338 [00:00<?, ?it/s]

In [25]:
calculate_accuracy(input_df=df,
                   feature_extractor=madde_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 1338
Dogru Tahmin Sayisi: 1303
Yanlis Tahmin Sayisi: 35
Basari Orani: 0.9738


In [26]:
show_false_predictions(input_df=df,
                       feature_extractor=madde_sayisi_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=False,
                       first_n=30)

,madde_sayisi,madde_sayisi_pred,baslik,url,kategori,data_text
3,11,16.0,YÜKSEKÖĞRETİM KURUMLARI TEŞKİLATI KANUNU İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2015/04/20150423-1.htm,Kanun,Resmî Gazete Tarihi: 23.04.2015 Resmî Gazete Sayısı: 29335\nKanun No. 6640 Kabul Tarihi : 31/03/2015\nYÜKSEKÖĞRETİM KURUMLARI TEŞKİLATI KANUNU İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 28/3/198...
5,12,15.0,İŞ KANUNU İLE TÜRKİYE İŞ KURUMU KANUNUNDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2016/05/20160520-24.htm,Kanun,Resmî Gazete Tarihi: 20.05.2016 Resmî Gazete Sayısı: 29717\nKanun No. 6715 Kabul Tarihi : 06/05/2016\nİŞ KANUNU İLE TÜRKİYE İŞ KURUMU KANUNUNDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 22/5/2003 tarihli ve 4857 sayılı İş Kanununun 7 nci mad...
16,44,45.0,TÜRK SİLÂHLI KUVVETLERİ PERSONEL KANUNU İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2021/03/20210305-12.htm,Kanun,Resmî Gazete Tarihi: 05.03.2021 Resmî Gazete Sayısı: 31414\nKanun No. 7281 Kabul Tarihi : 18/02/2021\nTÜRK SİLÂHLI KUVVETLERİ PERSONEL KANUNU İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 11/4/1928 tarihli ve 1219 sayılı Taba...
18,15,16.0,AMME ALACAKLARININ TAHSİL USULÜ HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2021/04/20210422-1.htm,Kanun,Resmî Gazete Tarihi: 22.04.2021 Resmî Gazete Sayısı: 31462\nKanun No. 7316 Kabul Tarihi : 15/04/2021\nAMME ALACAKLARININ TAHSİL USULÜ HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 21/7/1953 tarihli ve 6183 sayı...
24,38,39.0,BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2021/12/20211225-1.htm,Kanun,Resmî Gazete Tarihi: 25.12.2021 Resmî Gazete Sayısı: 31700\nKanun No. 7346 Kabul Tarihi : 21/12/2021\nBAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 5/12/1951 tarihli ve 5846 sayılı Fikir ve Sanat Eserleri Kanununun 72 nci maddesi...
31,4,5.0,DEVLET İSTİHBARAT HİZMETLERİ VE MİLLİ İSTİHBARAT TEŞKİLATI KANUNUNDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,http://www.resmigazete.gov.tr/eskiler/2012/02/20120218.htm,Kanun,Resmî Gazete Tarihi: 18.02.2012 Resmî Gazete Sayısı: 28208\nKanun No. 6278 Kabul Tarihi : 17/02/2012\nDEVLET İSTİHBARAT HİZMETLERİ VE MİLLİ İSTİHBARAT TEŞKİLATI KANUNUNDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 1/11/1983 tarihli ve 2937 sa...
33,27,32.0,SEÇİMLERİN TEMEL HÜKÜMLERİ VE SEÇMEN KÜTÜKLERİ HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,http://www.resmigazete.gov.tr/eskiler/2012/05/20120518.htm,Kanun,Resmî Gazete Tarihi: 18.05.2012 Resmî Gazete Sayısı: 28296\nKanun No. 6304 Kabul Tarihi : 09/05/2012\nSEÇİMLERİN TEMEL HÜKÜMLERİ VE SEÇMEN KÜTÜKLERİ HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 26/4/1961 tarih...
34,11,16.0,CEZA VE GÜVENLİK TEDBİRLERİNİN İNFAZI HAKKINDA KANUN İLE DENETİMLİ SERBESTLİK VE YARDIM MERKEZLERİ İLE KORUMA KURULLARI KANUNUNDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,http://www.resmigazete.gov.tr/eskiler/2012/04/20120411-11.htm,Kanun,Resmî Gazete Tarihi: 11.04.2012 Resmî Gazete Sayısı: 28261\nKanun No. 6291 Kabul Tarihi : 05/04/2012\nCEZA VE GÜVENLİK TEDBİRLERİNİN İNFAZI HAKKINDA KANUN İLE DENETİMLİ SERBESTLİK VE YARDIM MERKEZLERİ İLE KORUMA KURULLARI KANUNUNDA DEĞİŞİKLİK YAP...
37,81,82.0,ASKERLİK KANUNU İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,http://www.resmigazete.gov.tr/eskiler/2012/06/20120603.htm,Kanun,Resmî Gazete Tarihi: 03.06.2012 Resmî Gazete Sayısı: 28312\nKanun No. 6318 Kabul Tarihi : 22/05/2012\nASKERLİK KANUNU İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 21/6/1927 tarihli ve 1111 sayılı Askerlik Kanununun 4 üncü ma...
38,6,9.0,YÜKSEKÖĞRETİM KURUMLARI TEŞKİLATI KANUNUNDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,http

### MEVZUAT NO

In [16]:
mevzuat_no_extractor = MevzuatNoExtractor()

Yarisma submission modu: KAPALI


In [17]:
mevzuat_no_extractor.extract(df)

'mevzuat_no' cikariliyor... -> 'mevzuat_no_pred'


  0%|          | 0/1723 [00:00<?, ?it/s]

In [18]:
calculate_accuracy(input_df=df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=False)

Veri Sayisi: 1091
Dogru Tahmin Sayisi: 1067
Yanlis Tahmin Sayisi: 24
Basari Orani: 0.978


In [19]:
show_false_predictions(input_df=df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "kategori", "url", "kategori", "data_text"],
                       include_na=False,
                       first_n=40)

,mevzuat_no,mevzuat_no_pred,baslik,kategori,url,kategori,data_text
917,1998/5,1998/47504,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/5",Genelge,http://www.gib.gov.tr/node/89371,Genelge,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/47504\n\nTarih 15/12/1998\n\nSayı B.07.0.GEL.0.44/4412-17-047504\n\nKapsam \n\nT.C\n\nMALIYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.44/4412-17-047504 \n\nKONU:GELİR VERGİSİ KANU...
922,2006/16,5084,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",Genelge,https://kms.kaysis.gov.tr/Home/Goster/106310,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSosyal Sigortalar Kurumu Başkanlığı (Devredilen)\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B.13.2.SSK.5.01.08.00/VIII-031-502522 ANKARA\nKONU : Yatırımların ve istihdam...
923,2005/16-358,506,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",Genelge,https://kms.kaysis.gov.tr/Home/Goster/128284,Genelge,T.C.\nSOSYAL SİGORTALAR KURUMU BAŞKANLIĞI \n\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B. 13.2.SSK.5.01.08.00/VIII-031-946961 ANKARA\nKONU : ihale Konusu işlerden Kaynaklanan 06/12/2005\n\nBorçların Takibi\n\nGEN...
1072,2009/05,2009/5,SOSYAL GÜVENLİK KURUMU 2009/05 SAYILI GENELGESİ (İsteğe Bağlı Sigortalılık İşlemleri),Genelge,http://www.sgk.gov.tr/wps/wcm/connect/1ae26a23-473b-4cd9-9656-48f940832704/2009_05+iste%C4%9Fe+ba%C4%9Fl%C4%B1+sigortal%C4%B1l%C4%B1k+i%C5%9Flemleri.pdf?MOD=AJPERES&CACHEID=1ae26a23-473b-4cd9-9656-48f940832704,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI\n\nSosyal Sigortalar Genel Müdürlüğü\nSigortalı Tescil ve Hizmet Daire Başkanlığı\n\nAdres : Mithatpaşa Cad. No: 7 Sıhhiye/ANKARA Ayrıntılı bilgi için irtibat : Başar HALICI- Dai.Bşk \nTelefon : 0 312 431 6...
1105,2001/4,2001/14,"Tapu ve Kadastro Genel Müdürlüğü, 2001/14 Sayılı ""Bölge Arşivleri"" Konulu Genelge",Genelge,http://www.tkgm.gov.tr/tr/icerik/200114-sayili-bolge-arsivleri-konulu-genelge,Genelge,"Tapu ve Kadastro Genel Müdürlüğü, 2001/14 Sayılı ""Bölge Arşivleri"" Konulu Genelge \n\nT.C. BAŞBAKANLIK \n\nTAPU VE KADASTRO GENEL MÜDÜRLÜĞÜ \n\nTapu Arşiv Dairesi Başkanlığı \n\nSayı : B021TKG0150006-4109 Ankara \n\nKonu: \n\nGenelge \n\nTAPU VE ..."
1135,2017/12,2011/62,"SOSYAL GÜVENLİK KURUMU, FATURA BEDELLERİNİN ÖDENMESİ GENELGESİNİN YÜRÜRLÜKTEN KALDIRILMASINA DAİR 2017/12 SAYILI GENELGE",Genelge,https://kms.kaysis.gov.tr/Home/Goster/104268,Genelge,sosyal güve ylik koromu\n\nTC.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nGenel Sağlık Sigortası Genel Müdürlüğü\n\nSayı : 35837838/226 16/03/2017\nKonu : Fatura Bedellerinin Ödenmesi Genelgesinin \n\nYürürlükten Kaldırılmasına Dair Genelge\n\nGENELG...
1138,2017/09,2017/9,"SOSYAL GÜVENLİK KURUMU, ASGARİ ÜCRET DESTEĞİ HAKKINDA 27/02/2017 TARİHLİ VE 2017/09 SAYILI GENELGE",Genelge,https://kms.kaysis.gov.tr/Home/Goster/96578,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 24010506-010.06-E. 1105696 27/02/2017\nKonu : Asgari Ücret Desteği\n\nGENELGE\n2017-9\n\n6770 sayılı Türkiye Cumhuriyeti Emekli Sandığı Kanunu ile Bazı Kanun v...
1139,1046330,2017/1,"SOSYAL GÜVENLİK KURUMU, 2017/1 SAYILI GENELGE",Genelge,https://kms.kaysis.gov.tr/Home/Goster/105981,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nStrateji Geliştirme Başkanlığı\n\ni rSayı\nU\n\n32945953-010.06.01-E. 1046330 23/02/2017\nKonu-: Strateji Geliştirme Başkanlığı Daire Başkanlıktan \n\nve Alt Birimlerin Görev Tanımlan\ni\n\nİÇ GENELGE \...
1140,2017/07,2017/7,"SOSYAL GÜVENLİK KURUMU, 2017/7 SAYILI GENELGE SİGORTA PRİMLERİNİN ERTELENMESİ",Genelge,https://kms.kaysis.gov.tr/Home/Goster/96320,Genelge,"T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 89075644-202.99-E.695085 07/02/2017\nKonu : Sigorta primlerinin ertelenmesi\n\n

In [20]:
constants.COLS_MEVZUAT_NO

['Kanun',
 'Kanun Hükmünde Kararname',
 'Genelge',
 'Cumhurbaşkanlığı Kararnamesi',
 'Tüzük',
 'Yönetmelik']

In [21]:
filtered_df = filter_dataframe(input_df=df,
                               selected_categories=[],
                               custom_filter_array=[(~df.kategori.str.contains("Yönetmelik"))]
                              )

calculate_accuracy(input_df=filtered_df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=True)

show_false_predictions(input_df=filtered_df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=True,
                       first_n=20)

Veri Sayisi: 1126
Dogru Tahmin Sayisi: 1078
Yanlis Tahmin Sayisi: 48
Basari Orani: 0.9574


,mevzuat_no,mevzuat_no_pred,baslik,url,kategori,data_text
917,1998/5,1998/47504,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/5",http://www.gib.gov.tr/node/89371,Genelge,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/47504\n\nTarih 15/12/1998\n\nSayı B.07.0.GEL.0.44/4412-17-047504\n\nKapsam \n\nT.C\n\nMALIYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.44/4412-17-047504 \n\nKONU:GELİR VERGİSİ KANU...
922,2006/16,5084,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/106310,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSosyal Sigortalar Kurumu Başkanlığı (Devredilen)\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B.13.2.SSK.5.01.08.00/VIII-031-502522 ANKARA\nKONU : Yatırımların ve istihdam...
923,2005/16-358,506,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/128284,Genelge,T.C.\nSOSYAL SİGORTALAR KURUMU BAŞKANLIĞI \n\nSigorta İşleri Genel Müdürlüğü Sigorta Primleri Daire Başkanlığı\n\nSAYI : B. 13.2.SSK.5.01.08.00/VIII-031-946961 ANKARA\nKONU : ihale Konusu işlerden Kaynaklanan 06/12/2005\n\nBorçların Takibi\n\nGEN...
1025,undefined,95/6430,"GELİR İDARESİ BAŞKANLIĞI, YOLCU VE YÜK TAŞIMACILIĞI FAALİYETİNDE GÖTÜRÜLÜK DERECESİNE İLİŞKİN GENEL YAZI",http://www.gib.gov.tr/node/89455,Genelge,YOLCU VE YÜK TAŞIMACILIĞI FAALİYETİNDE GÖTÜRÜLÜK DERECESİNE İLİŞKİN GENEL YAZI\n\nTarih 17/04/1995\nSayı B.07.0.GEL.0.44/4412-1-20248\n\nKapsam \nT.C.MALİYE BAKANLIĞI\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.44/4412-1-20248\n\nKONU : YOLC...
1026,undefined,3218,"GELİR İDARESİ BAŞKANLIĞI, GÜMRÜK İDARELERİ TARAFINDAN KAĞIT ORTAMINDA GÜMRÜK BEYANNAMESİ DÜZENLENMESİNE VE KAMU KURUMLARININ GÜMRÜK BEYANNAMESİ ÖRNEKLERİNİN KAĞIT ORTAMDA VE ISLAK İMZALI İBRAZINA YÖNELİK TALEPLERİNİN KARŞILANMASINA SON VERİLMESİN...",http://www.gib.gov.tr/node/89445,Genelge,GÜMRÜK İDARELERİ TARAFINDAN KAĞIT ORTAMINDA GÜMRÜK BEYANNAMESİ DÜZENLENMESİNE VE KAMU KURUMLARININ GÜMRÜK BEYANNAMESİ ÖRNEKLERİNİN KAĞIT ORTAMDA VE ISLAK İMZALI İBRAZINA YÖNELİK TALEPLERİNİN KARŞILANMASINA SON VERİLMESİNE İLİŞKİN GENEL YAZI\n\nTa...
1027,undefined,3568,"GELİR İDARESİ BAŞKANLIĞI, 346 SIRA NO'LU VERGİ USUL KANUNU GENEL TEBLİĞİ İLE İLGİLİ GENEL YAZI",http://www.gib.gov.tr/node/89469,Genelge,346 SIRA NO'LU VERGİ USUL KANUNU GENEL TEBLİĞİ İLE İLGİLİ GENEL YAZI\n\nTarih 14/03/2005\n\nSayı B.07.0.GEL.0.71/7103-275 /12360\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.71/7103-275 /12360\n\n........
1031,undefined,6183,"GELİR İDARESİ BAŞKANLIĞI, BELİRTİLEN SÜRE İÇERİSİNDE TECİL VE TAKSİTLENDİRME TALEBİNDE BULUNANLARA İLİŞKİN GENEL YAZI",http://www.gib.gov.tr/node/89477,Genelge,BELİRTİLEN SÜRE İÇERİSİNDE TECİL VE TAKSİTLENDİRME TALEBİNDE BULUNANLARA İLİŞKİN GENEL YAZI\n\nTarih 08/01/2002\nSayı B.07.0.GEL.0.38/3802-43 / 001089\n\nKapsam \nT.C.MALİYE BAKANLIĞI\nGelirler Genel Müdürlüğü\n\nTARİHİ : 08.01.2002\nSAYISI : B.0...
1032,undefined,213,"GELİR İDARESİ BAŞKANLIĞI, İZMİR'DE MUCBİR SEBEP HALİNE İLİŞKİN GENEL YAZI",http://www.gib.gov.tr/node/89492,Genelge,İZMİR'DE MUCBİR SEBEP HALİNE İLİŞKİN GENEL YAZI\n\nTarih 21/12/2001\n\nSayı B.07.0.GEL.0.28/2810-13-866-068334\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI: B.07.0.GEL.0.28/2810-13-866-068334\n\nİZMİR VALİLİĞİ (Defte...
1033,undefined,213,"GELİR İDARESİ BAŞKANLIĞI, PAY DEVRİ veya DEVİR VAADİ SÖZLEŞMELERİNİN NOTERLERCE TASDİKİNE İLİŞKİN GENEL YAZI",http://www.gib.gov.tr/node/89454,Genelge,PAY DEVRİ veya DEVİR VAADİ SÖZLEŞMELERİNİN NOTERLERCE TASDİKİNE İLİŞKİN GENEL YAZI\n\nTarih 29/01/2001\n\nSayı B.07.0.GEL.0.71/7103-140\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.71/7103-140\n\n……………...
1035,undefined,2002/4930,"GELİR

In [22]:
print(df.loc[1049]["data_text"])

MURİSİN TÜRKİYE’DEKİ İKAMETGÂH ADRESİNİN BULUNMAMASI VEYA TESPİT EDİLEMEMESİ HALİNDE, TARHA YETKİLİ VERGİ DAİRESİNİN BELİRLENMESİ HAKKINDA GENEL YAZI

Tarih 15/08/2011

Sayı B.07.1.GİB.0.06.60.010.07.03[1]/78258

Kapsam 

T.C.

MALİYE BAKANLIĞI

Gelir İdaresi Başkanlığı

SAYI :B.07.1.GİB.0.06.60.010.07.03[1]

KONU :

......................VERGİ DAİRESİ BAŞKANLIĞINA

.......................VALİLİĞİNE

(Defterdarlık: Gelir Müdürlüğü)

Bilindiği üzere, 7338 sayılı Veraset ve İntikal Vergisi Kanununun 6 ncı maddesinin (c) fıkrasında, muris veya tasarrufu yapan şahsın Türkiye'de hiç ikamet etmediği veya son ikametgâhı tespit olunamadığı durumda veraset ve intikal vergisinin Maliye Bakanlığının tayin edeceği yer vergi dairesince tarh olunacağı hükme bağlanmıştır.

Mükelleflere kolaylık sağlanması bakımından, murisin Türkiye'deki ikametgâh adresinin bulunmaması veya tespit edilememesi halinde, tarha yetkili vergi dairesinin belirlenmesi için mükellefler tarafından Maliye Bakanlığı (Gelir İdar